In [1]:
import context
import numpy as np
import pandas as pd
import Fitting as Fit
import Gillespie as Glp

# initialise constants
species_lenser = ["x","y","z"]  # z inner und x free, y loosely bound
# loss side
L_lenser = pd.DataFrame({"reaction01":[0, 0, 0],  # außen - x
            "reaction02":[1, 0, 0],  # x - y
            "reaction03":[0, 1, 0],  # y - z
            "reaction11":[1, 0, 0],  # x - außen
            "reaction12":[0, 1, 0],  # y - x
            "reaction13":[0, 0, 1]},  # z - y
            index = species_lenser)
# gain side
R_lenser = pd.DataFrame({"reaction01":[1, 0, 0],  # außen - x
            "reaction02":[0, 1, 0],  # x - y
            "reaction03":[0, 0, 1],  # y - z
            "reaction11":[0, 0, 0],  # x - außen
            "reaction12":[1, 0, 0],  # y - x
            "reaction13":[0, 1, 0]},  # z - y
            index = species_lenser)
N_lenser = R_lenser - L_lenser

# number of fluorescent molecules after bleach
startQuantities_lenser = {"x":[0],"y":[0],"z":[0]}

time_limit_lenser = 1160.0
runs_lenser = 2
# colours for plotting
colours_lenser = {"x":"red", "y":"forestgreen", "z":"lightblue"}
# component of the measured FRAP data for fitting
pml_component = "PML I WT"

measurement = pd.read_csv("./../Daten/FRAP_comparison_all_new_AssemblyDynamicsOfPMLNuclearBodiesInLivingCells_cleaned.csv", index_col="time[s]")
measured_times = measurement.index.tolist()
time_intervals = Fit.divide_time_axis_equidistantly(measured_times)

def fitness(parameters):
    # simulate one or more Gillespie runs
    trajectory = Glp.monte_carlo_gillespie(parameters, L_lenser, N_lenser, startQuantities_lenser, runs=runs_lenser, time_max=time_limit_lenser)
    # list of tuples of gillespie times_list and added_quantities_list for output signal
    list_of_output_data = Glp.make_output_signal(trajectory, species_lenser)
    # DataFrame of averaged gillespie data per time interval, time intervals as index
    assigned_simulation_data = Fit.assign_simulation_times_to_time_ranges_average(time_intervals,measured_times,list_of_output_data)
    # normalize the simulation data
    norm_value = 200  # max 200 molecules in a ROI
    normalized_assigned_simulation_data = assigned_simulation_data/norm_value
    # calculate the absolute differences between the simulated and measured data
    differences = Fit.calculate_differences(measurement[pml_component], normalized_assigned_simulation_data)
    # create a quality score by adding up the individual runs and averaging the sums
    quality_of_fitness = np.mean(differences.sum())
    
    return quality_of_fitness

ret = fitness({"reaction01": 1,
             "reaction02": 1,
             "reaction03": 1,
             "reaction11": 0.1,
             "reaction12": 1,
             "reaction13": 1})


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [2]:
print(ret)

26.76257768378471
